# Webscrapping

## Webscrapping basique avec unique package beautifulsoup

### Installation des packages

In [ ]:
!pip install lxml

import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint

##### Dans un premier temps, on chercher à récupérer des informations sur la communication verte de l'entreprise Pfizer. Pour cela, on test le webscrapping sur différents sites internet : CNN, Europe PMC et Facebook

### CNN 

In [12]:
nom_entreprise = "pfizer"
url = "https://edition.cnn.com/search?q={0}&from=0&size=10&page=1&sort=newest&types=all&section=".format(nom_entreprise)

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
        
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")

search_results_count_div = soup.find('div', class_='search__results-count')
search_results_count_div


<div class="search__results-count">
<strong id="search__query"></strong>
</div>

##### Conclusion : Aucun texte présent dans la balise recherchée...

### EUROPE PMC

In [5]:
from bs4 import BeautifulSoup
import requests

numero_article = 10677964
url = "https://europepmc.org/article/PMC/PMC{0}".format(numero_article)

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Récupérez la balise <h2> avec l'attribut id="article--free-full-text--header"
    h2_tag = soup.find('h2', {'id': 'article--free-full-text--header'})
    
    if h2_tag:
        print("Balise <h2> trouvée :", h2_tag.text)
    else:
        print("Balise <h2> non trouvée.")
    
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")


Balise <h2> non trouvée.


##### Conclusion : On ne trouve pas la basise qui permet d'afficher l'abstract de l'article

### Facebook

In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://www.facebook.com/Pfizer/?locale=fr_FR"

response = requests.get(url)

if response.status_code == 200:
    # Utilisez response.text pour obtenir le contenu sous forme de chaîne de caractères
    html_content = response.text

    # Utilisez BeautifulSoup avec html_content
    soup = BeautifulSoup(html_content, 'html.parser')
    all_post = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z') 
    
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")


all_post


[]

##### Conclusion : On ne trouve pas les balises correspondant aux postes

##### Possible problème de la méthode utilisée précedemment : les sites internet recherchés sont dynamiques. Pour résoudre ce problème, on essaye le package selinium.

## Webscrapping avec selinium

### Installation des packages

In [7]:
!sudo apt-get update
!sudo apt install -y unzip xvfb libxi6 libgconf-2-4 -y
!sudo apt install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()


!pip install webdriver_manager

import selenium
from webdriver_manager.chrome import ChromeDriverManager

path_to_web_driver = ChromeDriverManager().install()


import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--verbose') 


from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(service=service,
                           options=chrome_options)

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                   
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease   
Hit:6 https://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease
Fetched 229 kB in 2s (98.9 kB/s)                    
Reading package lists... Done
W: https://apt.postgresql.org/pub/repos/apt/dists/jammy-pgdg/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libxi6 is already the newest version (2:1.8-1build1).
libgconf-2-4 is already the newest version (3.2.6-7ubuntu2).
unzip is already the newest version (6.0-26ubuntu3.1).
xvfb is already the newest version (2

### CNN 

In [19]:
nom_entreprise = "pfizer"
url = "https://edition.cnn.com/search?q={0}&from=0&size=10&page=1&sort=newest&types=all&section=".format(nom_entreprise)

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')

soup


<html><head></head><body></body></html>

##### Conclusion : on fait pire qu'avec le package beautiful soup uniquement, on arrive pas à télécharger le code HTML

### Facebook

In [26]:
url = "https://www.facebook.com/Pfizer/?locale=fr_FR"

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')
div_tags = soup.find_all('div', class_='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z')

div_tags

[]

##### Conclusion : On arrive toujours pas à récupérer les postes

### Europe PMC

In [15]:
numero_article = 10677964
url = "https://europepmc.org/article/PMC/PMC{0}".format(numero_article)

browser.get(url)

# Obtenez le contenu de la page après que JavaScript ait pu mettre à jour le DOM
html_content = browser.page_source

# Utilisez BeautifulSoup comme précédemment
soup = BeautifulSoup(html_content, 'html.parser')
h2_tag = soup.find('h2', {'id': 'article--free-full-text--header'})

if h2_tag:
    print("Balise <h2> trouvée :", h2_tag.text)
else:
    print("Balise <h2> non trouvée.")

Balise <h2> non trouvée.


##### Conclusion : pas d'amélioration